In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from datetime import timedelta

In [2]:
#The latest date in the dataset is 2019-02-26. We will use this date as our current day.
today = pd.to_datetime('2019-02-26 00:00:00')

In [3]:
#Import the datasets
root_path = '/home/dqminhv/Springboard/Springboard_Capstone_Project_3/data/'
answers = pd.read_csv(root_path + 'answers.csv')
answer_scores = pd.read_csv(root_path + 'answer_scores.csv')
comments = pd.read_csv(root_path + 'comments.csv')
emails = pd.read_csv(root_path + 'emails.csv')
groups = pd.read_csv(root_path + 'groups.csv')
group_memberships = pd.read_csv(root_path + 'group_memberships.csv')
matches = pd.read_csv(root_path + 'matches.csv')
professionals = pd.read_csv(root_path + 'professionals.csv')
questions = pd.read_csv(root_path + 'questions.csv')
question_scores = pd.read_csv(root_path + 'question_scores.csv')
school_memberships = pd.read_csv(root_path + 'school_memberships.csv')
students = pd.read_csv(root_path + 'students.csv')
tags = pd.read_csv(root_path + 'tags.csv')
tag_questions = pd.read_csv(root_path + 'tag_questions.csv')
tag_users = pd.read_csv(root_path + 'tag_users.csv')

In [4]:
#Create a function to clean the text feature
stop=set(stopwords.words("english"))

def clean_text(text):
    """
    Cleans text for TF-IDF analysis.
    Args:
    text: A string containing the text to be cleaned.
    Returns:
    A cleaned string suitable for TF-IDF analysis.
    """
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove URLs
    text = re.sub(r'[^\s]+@.*?\s+', '', text)
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove underscores
    text = re.sub(r'_', '', text)
    # Remove single characters
    text = re.sub(r'\s+', ' ', text)
    # Remove multiple spaces
    text = text.strip()
    
    # Remove stop words
    text = " ".join([word for word in text.split() if word not in stop])
    
    # Lemmatizing
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text.lower()

In [5]:
#Convert some date time columns to datetime objects.
questions['questions_date_added'] = pd.to_datetime(questions['questions_date_added'])
professionals['professionals_date_joined'] = pd.to_datetime(professionals['professionals_date_joined'])
emails['emails_date_sent'] = pd.to_datetime(emails['emails_date_sent'])
answers['answers_date_added'] = pd.to_datetime(answers['answers_date_added'])

In [6]:
questions['questions_date_added'].describe()

count                                  23931
mean     2017-01-07 08:05:27.113660160+00:00
min                2011-09-27 15:23:17+00:00
25%         2016-05-16 19:59:27.500000+00:00
50%                2016-10-31 22:37:57+00:00
75%         2018-01-24 22:38:08.500000+00:00
max                2019-01-31 22:32:00+00:00
Name: questions_date_added, dtype: object

In [7]:
answers['answers_date_added'].describe()

count                                  51123
mean     2017-03-11 20:38:32.033937920+00:00
min                2011-10-05 20:33:04+00:00
25%         2016-06-11 21:03:31.500000+00:00
50%                2017-05-10 16:51:04+00:00
75%         2018-04-04 12:07:17.500000+00:00
max                2019-01-31 23:47:27+00:00
Name: answers_date_added, dtype: object

In [8]:
professionals['professionals_date_joined'].describe()

count                                  28152
mean     2017-06-18 23:49:24.520460288+00:00
min                2011-10-05 20:35:19+00:00
25%      2016-06-08 10:23:41.249999872+00:00
50%         2017-11-02 13:37:35.500000+00:00
75%         2018-08-01 21:45:05.500000+00:00
max                2019-01-31 23:59:09+00:00
Name: professionals_date_joined, dtype: object

# Recommnedation strategy


* Create a profile for each question by its content and its tags.
* Split the questions and professionals datasets into train/test set.
* Use questions/professionals before 2017-05-10 as train sets, and after 2017-05-10 as test sets.
* Create a profile for each professional by their tags and the questions' content their answered before.
* Using cosine similarity, we calculate the similarity score of each question in the test set to the list of professionals.
* We assign each professionals with 10 highest similarity score questions.
* We estimate the recommemdation power of the model by comparing the response rate in the test set and the response rate by the recommendation model.

# Preparing data for modeling

In [9]:
question_w_answer = pd.merge(questions, answers,
                                 left_on='questions_id', 
                                 right_on='answers_question_id', 
                                 how='right').drop('answers_question_id', axis=1)

In [10]:
question_w_answer_set = set(question_w_answer['questions_id'].values)

In [11]:
#Combine tags names to the questions dataframe
question_tag = pd.merge(tag_questions, tags,
                        left_on='tag_questions_tag_id', 
                        right_on='tags_tag_id', 
                        how='left').drop('tags_tag_id', axis=1)

#Rename the columns of the new dataframe
question_tag.rename(columns={'tag_questions_tag_id':'tag_id',
                             'tag_questions_question_id':'question_id',
                             'tags_tag_name':'tag_name'}, inplace=True)

In [12]:
#Create a dataframe with each question id acssociate with a list of its tags names
question_tag_list = question_tag.groupby("question_id")["tag_name"].agg(lambda x: ",".join(x)).reset_index()

## Training data

In [13]:
#Use the day at 50th percentile of answers_date_added column in answers data_frame
cutoff_day = pd.to_datetime('2017-05-10 16:51:04+00:00')

### Questions with answer(s)

In [14]:
train_answer = answers[answers['answers_date_added'] < cutoff_day].copy(deep=True)

In [15]:
train_answer.rename(columns={'answers_id':'answer_id',
                            'answers_author_id':'professional_id',
                            'answers_question_id':'question_id',
                            'answers_date_added':'answer_date_added',
                            'answers_body':'answer_body'},inplace=True)

In [16]:
train_question = questions[questions['questions_id'].\
                           isin(set(train_answer['question_id'].values))].\
                            reset_index(drop=True)

In [17]:
train_question.rename(columns={'questions_id':'question_id',
                              'questions_author_id':'user_id',
                              'questions_date_added':'question_date_added',
                              'questions_title':'question_title',
                              'questions_body':'question_body'}, inplace=True)

In [18]:
train_answer_score = pd.merge(train_answer, answer_scores,
                              left_on='answer_id', 
                              right_on='id', 
                              how='left').drop('id', axis=1)

train_question_answer = pd.merge(train_question, train_answer_score,
                                 left_on='question_id', 
                                 right_on='question_id', 
                                 how='right')

In [19]:
#Create a set of questions that have at least one answers
train_question_w_answer_set = set(train_question_answer['question_id'].values)

In [20]:
#Create a set of professionals that answered at least one question
train_professional_w_answer_set = set(train_answer['professional_id'].values)

### Questions profiles in the training set

#### Questions content

In [21]:
#Create a train question dataframe with only questions with answers
train_question = train_question[train_question["question_id"].isin(train_question_w_answer_set)]

In [22]:
#Combine questions_title and questions_body as questions_content
train_question['question_content'] = ""
train_question['question_content'] = train_question['question_title'] + " " + train_question['question_body']
train_question.drop(['question_title', 'question_body'], axis=1, inplace=True)

In [23]:
train_question.reset_index(drop=True, inplace=True)

In [24]:
#Clean the questions_content and tags for tf-idf
train_question['question_content'] = train_question['question_content'].apply(clean_text)

#### Adding tags names to the questions in training set

In [25]:
#Combine the tags names list of each question to the training question data
train_question = pd.merge(train_question, question_tag_list,
                         left_on='question_id',
                         right_on='question_id',
                         how='left')

In [26]:
train_question['tag_name'].fillna('', inplace=True)

### Professionals profiles in the training set

#### Adding tags names to the professional training set

In [27]:
#Create a professional training dataframe
#This includes professionals joined in before the cutoff day, and answered at least 1 question.
train_professional = professionals.copy(deep=True)
train_professional = train_professional[train_professional["professionals_id"].isin(train_professional_w_answer_set)]

In [28]:
train_professional.rename(columns={'professionals_id':'professional_id',
                                   'professionals_date_joined':'professional_date_joined'},
                          inplace=True)

In [29]:
train_professional.drop(['professionals_location', 'professionals_industry', 'professionals_headline'], axis=1, inplace=True)

In [30]:
#Combine tags names to the tag_users
user_tag = pd.merge(tag_users, tags,
                    left_on='tag_users_tag_id', 
                    right_on='tags_tag_id', 
                    how='left').drop('tags_tag_id', axis=1)

In [31]:
user_tag.rename(columns={'tag_users_tag_id':'tag_id',
                        'tag_users_user_id':'user_id',
                        'tags_tag_name':'tag_name'},
               inplace=True)

In [32]:
user_tag_list = user_tag.groupby("user_id")["tag_name"].agg(lambda x: ",".join(x)).reset_index()

In [33]:
train_professional = train_professional.merge(user_tag_list,
                                              left_on='professional_id',
                                              right_on='user_id',
                                              how='left').drop('user_id', axis=1)

In [34]:
train_professional['tag_name'].fillna('no_tag', inplace=True)

#### Professionals profiles with previous questions content

In [35]:
#Merge the 
train_question_w_answer = pd.merge(train_answer, train_question,
                                        left_on='question_id',
                                        right_on='question_id',
                                        how='left')

In [36]:
train_professional_question = train_question_w_answer.\
                                            groupby("professional_id")["question_content"].\
                                            agg(lambda x: ",".join(x)).reset_index()

In [37]:
train_professional_profile = pd.merge(train_professional, train_professional_question,
                                          left_on='professional_id',
                                          right_on='professional_id',
                                          how='left')

In [38]:
train_professional_profile.fillna('', inplace=True)

In [39]:
train_professional_profile.to_csv('train_data.csv', index=False, sep=',', header=True)

## Testing data

### Questions with answer(s) in the testing set

In [40]:
test_answer = answers[answers['answers_date_added'] >= cutoff_day].copy(deep=True)

In [41]:
test_question = questions[questions['questions_id'].isin(set(test_answer['answers_question_id'].values))].reset_index(drop=True)

In [42]:
test_answer_score = pd.merge(test_answer, answer_scores,
                              left_on='answers_id', 
                              right_on='id', 
                              how='left').drop('id', axis=1)

test_question_answer = pd.merge(test_question, test_answer_score,
                                 left_on='questions_id', 
                                 right_on='answers_question_id', 
                                 how='right').drop('answers_question_id', axis=1)

In [43]:
#Create a set of questions that have at least one answers
test_question_w_answer_set = set(test_question_answer['questions_id'].values)

In [44]:
#Create a set of professionals that answered at least one question
test_professional_w_answer_set = set(test_answer['answers_author_id'].values)

In [45]:
test_answer.rename(columns={'answers_id':'answer_id',
                            'answers_author_id':'professional_id',
                            'answers_question_id':'question_id',
                            'answers_date_added':'answer_date_added',
                            'answers_body':'answer_body'
                           },inplace=True)

### Questions profiles in the test set

#### Questions content

In [46]:
#Combine questions_title and questions_body as questions_content
test_question['question_content'] = ""
test_question['question_content'] = test_question['questions_title'] + " " + test_question['questions_body']
test_question.drop(['questions_title', 'questions_body'], axis=1, inplace=True)

In [47]:
#Clean the questions_content and tags for tf-idf
test_question['question_content'] = test_question['question_content'].apply(clean_text)

In [48]:
#Rename columns with appropriate names
test_question.rename(columns={'questions_id':'question_id',
                              'questions_date_added':'question_date_added',
                              'questions_author_id':'question_author_id'},
                      inplace=True)

#### Adding tags names to the questions in test set

In [49]:
#Combine the tags names list of each question to the training question data
test_question = pd.merge(test_question, question_tag_list,
                         left_on='question_id',
                         right_on='question_id',
                         how='left')

In [50]:
test_question['tag_name'].fillna('', inplace=True)

In [51]:
test_question.rename(columns={'question_author_id':'user_id'},inplace=True)

### Professionals profiles in the test set

#### Adding tags names to the professional test set

In [52]:
#Create a professional test dataframe
test_professional = professionals.copy(deep=True)
test_professional = test_professional[test_professional["professionals_id"].\
                                      isin(test_professional_w_answer_set)]

In [53]:
test_professional.rename(columns={'professionals_id':'professional_id',
                                   'professionals_date_joined':'professional_date_joined'},
                          inplace=True)

In [54]:
test_professional.drop(['professionals_location', 'professionals_industry', 'professionals_headline'], axis=1, inplace=True)

In [55]:
test_professional = test_professional.merge(user_tag_list,
                                            left_on='professional_id',
                                            right_on='user_id',
                                            how='left').drop('user_id', axis=1)

In [56]:
test_professional['tag_name'].fillna('no_tag', inplace=True)

#### Professionals profiles with previous questions content

In [57]:
#Merge the 
test_question_w_answer = pd.merge(test_answer, test_question,
                                        left_on='question_id',
                                        right_on='question_id',
                                        how='left')

In [58]:
test_professional_question = test_question_w_answer.\
                                            groupby("professional_id")["question_content"].\
                                            agg(lambda x: ",".join(x)).\
                                            reset_index()

In [59]:
test_professional_profile = pd.merge(test_professional, test_professional_question,
                                          left_on='professional_id',
                                          right_on='professional_id',
                                          how='left')

In [60]:
test_professional_profile.fillna('', inplace=True)

In [61]:
test_professional_profile.to_csv('test_data.csv', index=False, sep=',', header=True)